## Collateralized Debt Obligation

A Collateralized Debt Obligation (CDO) is a credit derivative where the issuer, typically investment banks, gather risky assets and repackage them into discrete classes (*tranches*) based on the level of credit risk assumed by the investor. These tranches of securities become the final investment product.

Tranches are named to reflect their risk profile: senior, mezzanine and subordinated/equity and are delimited by the attachment ($L$) and detachment points ($U$), which represent the percentages of the total principal defining their boundaries. 
For example, a 5-10% tranche has an attachment point of 5% and a detachment point of 10%. 

<!----When the accumulated loss of the reference pool is no more than 5% of the total initial notional of the pool, the tranche will not be affected. However, when the loss has exceeded 5%, any further loss will be deducted from the tranche's notional until the detachment point, 10%, is reached.-->

Each of these tranches has a different level of seniority relative to the others in the sense that a senior tranche has coupon
and principal payment priority over a mezzanine tranche, while a mezzanine tranche has
coupon and principal payment priority over an equity tranche. 

Indeed they receive returns using a set of rules known as *waterfall*. Incomes of the portfolio are first used to provide returns to the most senior tranche, then to the next and so on.
So the senior tranches are generally safest because they have the first claim on the collateral, although they'll offer lower coupon rates.

It is important to note
that a CDO only redistributes the total risk associated with the underlying pool of assets
to the priority ordered tranches. It neither reduces nor increases the total risk associated
with the pool.

There are various kind of CDOs:

* in a **Cash CDO** the reference portfolio consists of corporate bonds owned by the CDO issuer. Cash flows from collateral are used to pay principal and interest to investors. If such cash flows prove inadequate, principal and interest is paid to tranches according to their seniority. 

<img src="cdo_structure.png">

* in a **Synthetic CDO** the underlying reference portfolio is no longer a physical portfolio of bonds or loans, instead it is a *fictitious* portfolio consisting of a number of names each with an associated notional amount. The value of a synthetic CDO usually comes from insurance premiums of credit default swaps paid for by investors. The seller assumes the underlying assets will perform while the investor, on the other hand, assumes the underlying assets will default.

### Cash CDO Expected Losses

Consider a Cash CDO with a maturity of 1 year, made of 125 bonds. Each bond pays a coupon of one unit after 1 year and it has not yet defaulted (the recovery rate $R$ is assumed 0). We are interested in the following three tranches: equity ([0, 3] defaults), mezzanine ([4, 6] defaults) and senior ([7, 9] defaults), (note that now tranches are identified through the number of defaults and not percentages of the principal). 

<img src="ex_cdo_1.png">

We also assume that the probability of default within 1 year are identical for each bond ($Q$) and that the correlation between each pair is also identical and equal to $\rho$.

Under these assumptions we are in the position to use the Gaussian Copula Model and the derivation of the expected losses results quite simple.

The probability of having $l$ defaults, conditional to the market parameter $M$ will follow a binomial distribution given by

$$p(l|M) = \binom{N}{l}Q_M^l (1-Q_M)^{N-l}$$

where $N$ is the number of bonds in the portfolio and 

$$Q_M = \Phi\left(\cfrac{\Phi^{-1}(Q)-\sqrt{\rho}M}{\sqrt{1-\rho}}\right)$$
where $\Phi$ is the standard normal CDF and $Q$ the probability of default within 1 year of a single name.

From the definition of each tranche with have that the expected losses are

* $\mathbb{E}_{\textrm{equity loss}}(l, Q_M)=3\cdot p(l\ge 3|M) + \sum_{k=1}^{2}k\cdot p(l=k|M)$
* $\mathbb{E}_{\textrm{mezzanine loss}}(l, Q_M)=3\cdot p(l\ge 6|M) + \sum_{k=1}^{2}k\cdot p(l=k+3|M)$
* $\mathbb{E}_{\textrm{senior loss}}(l, Q_M)=3\cdot p(l\ge 9|M) + \sum_{k=1}^{2}k\cdot p(l=k+6|M)$

Each probability $\mathbb{P}$ can be calculated by integrating the above with respect to $M$

$$ \mathbb{E}_{\mathrm{tranche}} = \int_{-\infty}^{\infty}{\mathbb{E}_{\mathrm{tranche}}(l, Q_M) f_M(m)dm} $$

Let's see the corresponding $\tt{python}$ implementation.
First we import the necessary modules and define the needed constants.

In [51]:
from scipy.stats import binom, norm 
from scipy.integrate import quad 
import numpy as np

N = 125 
Q = 0.02
C = 1
R = 0
tranches = [[1,3],[4, 6],[7,9]]

The we define a function $\tt{p}$ which implements the expected losses for each tranche.
The function depends on the parameter $\tt{M}$, and takes as inputs the correlation $\tt{rho}$ and the tranche attach-detach limits.

In [45]:
def one_factor_model(M, rho, lims):
    QM = norm.cdf((norm.ppf(Q)-np.sqrt(rho)*M)/np.sqrt(1-rho))
    pN = binom(N, QM)
    loss = 3 * (1 - pN.cdf(lims[1]-1))
    for i in range(lims[0], lims[1]):
        l = i - lims[0] + 1
        loss += l * pN.pmf(i)
    
    return norm.pdf(M)*loss

Finally we loop over a range of possible values for the correlation on each tranche to draw the plot of the expected losses vs the correlation.

In [50]:
res = [[], [], []]

for i in range(len(tranches)):
    for rho in np.arange(0, 1.10, 0.10): 
        if rho == 1.0:
            rho = 0.9999
        v = quad(one_factor_model, -np.inf, np.inf, 
                 args=(rho, tranches[i])) 
        res[i].append(v[0])

In [52]:
print (res)

[[2.0932572265621565, 1.7169402151018895, 1.443002683957591, 1.2171962594302363, 1.0206402810834037, 0.8441552229624638, 0.6823299000749955, 0.5313966601647971, 0.38798633219876355, 0.24700179024604607, 0.06338924905548826], [0.38845725736229153, 0.5400602571948848, 0.5442259316012925, 0.5136292996390736, 0.47016000780264683, 0.42040303085579434, 0.36661877474427984, 0.30926648241359367, 0.2474601501699685, 0.1777006656151882, 0.06264493837746456], [0.01800311179593733, 0.1644223954140962, 0.24505246417287352, 0.27725024370955487, 0.28395123245157244, 0.27550398879390176, 0.25676537674038025, 0.22989416138150914, 0.19518376393318326, 0.1501553932410937, 0.06227576677371002]]


<img src="losses_vs_rho.png">

Some considerations can be done from these results. First of all, as expected, the equity tranche is the riskier, producing the highest level of loss. The 

$$
\mathbb{E}_{\textrm{equity loss}}\ge \mathbb{E}_{\textrm{mezzanine loss}} \ge \mathbb{E}_{\textrm{senior loss}}
$$ 

relation holds only if each tranche has the same notional exposure (in our example 3).

Then we can notice that in the equity tranche losses are decreasing in $\rho$. When the correlation is low indeed the probability to have few defaults is higher than that of many. As the correlation increases, there will be more and more "simultaneous" defaults so also other tranches start to suffer losses. In the extreme case of correlation equal to 1 all the tranches are the same (indeed the expected losses curves join together). 

When considering all the tranches covering the entire number of names, the last tranche (the one with detachment point of 100\%) is always increasing in $\rho$. Again this can be explained with the correlated defaults. 
Also, the total expected losses on the three tranches is independent of $\rho$. This is not an accident but it is due to the fact that every default scenario is now categorized in one of the plotted tranches while before this was not the case.

## Synthetic CDO Valuation

Imagine a CDO made of $N$ names in the reference portfolio. The total notional of the protfolio is $F$.
When the $i^{th}$ name defaults, then the portfolio incurs in a loss of $F/N(1-R)$ (the recovery rate is assumed to be fixed for all entities of the portfolio).

The tranche loss function $TL^{L,U}(l)$ for a given time $t$ is a function of the number of defaults $l$ occurred up to that time and is given by

$$TL_{t}^{L,U}=\mathrm{max}(\mathrm{min}(l/N\cdot F(1-R), U)-L, 0)$$

where $l/N\cdot F(1-R)$ is the total portfolio loss, if it is greater than $U$ then the tranche loss is $U$. Conversely if it is lower than $L$ there is no loss.

So for example suppose $L=3\%$ and $U=7\%$ and suppose also that the portfolio loss is $5\%$. Then the tranche loss is 2\% of the total portfolio notional (or 50\% of the tranche notional $=7\%-3\%=4\%$).

When an investor *sells protection* on a tranche she is guaranteeing to reimburse any realized losses on the tranche to the *protection buyer*. To better understand this concept it is useful to think of the protection as an *insurance*. 

In return, the protection seller receives a premium at regular intervals (typically every three months) from the protection buyer.

### Premium Leg
As seen above the premium leg represents the payments that are done periodically by the protection buyer to the protection seller.

These payments are made at the end of each time interval and are proportional to the **remaining notional** in the tranche (this is an important difference with respect to CDS, where the contract ends as soon as a default occurs).

We can then write the NPV of the premium leg as

$$\mathrm{NPV}_{\mathrm{premium}}^{L,U}=S\sum^{n}_{i=1}D(d_i)\cfrac{(d_i - d_{i-1})}{360}\left(F(U-L)-\mathbb{E}[TL_{d-1}^{L,U}]\right)$$
where $n$ is the number of payment dates, $D(d_i)$ is the discount factor, $S$ is the annualized premium. The expected value represents the expected notional remaining in the tranche at time 
$d_{i-1}$.
Note that for simplicity we are ignoring that the default may take place at any time between each payment date.

### Default Leg
The default leg represents the cash flows paid to the protection buyer upon losses occurring in the considered tranche. 

The NPV of the leg can be expressed as
$$\mathrm{NPV}_{\mathrm{default}}^{L,U}=\sum_{i=1}^{n}D(d_i)\left(\mathbb{E}[TL_{d_i}^{L,U}]-\mathbb{E}[TL_{d_{i-1}}^{L,U}]\right)$$
where the argument in parenthesis is the expected losses between time $d_{i-1}$ up to $d_i$. 

Therefore the key ingredient for the valuation of a CDO is the calculation of $\mathbb{E}[TL_{d_i}^{L,U}]$ which appears in both legs.
Using the Gaussian copula it is relatively easy to compute it. 
Indeed we know that 

$$TL_{t}^{L,U}=\mathrm{max}(\mathrm{min}(l/N\cdot F(1-R), U)-L, 0)$$

where the only random variable is the number of defaults $l$. We also know that 

$$\mathbb{E}[TL_{t}^{L,U}] = \sum_{l=0}^{N}TL_{t}^{L,U}\cdot \int_{-\infty}^{\infty} Q(l_{t|M}=j) \phi(M)dM$$

And has we have already seen this calculation can be carried on without too much effort.
The large popularity of the Gaussian copula just resides in this, it allows to compute very quickly very complicated contracts like CDOs which usually involve a large number of correlated names.

### CDO Fair Value
The *fair value* of a CDO tranche is that value of the premium $S^*$ for which the expected value of the premium leg equals the expected value of the default leg and for what we have seen depends on the expected value of the tranche loss function.

$$ S^* = \cfrac{\mathrm{NPV_{default}}^{L,U}}{\sum^{n}_{i=1}D(d_i)\cfrac{(d_i - d_{i-1})}{360}\left(F(U-L)-\mathbb{E}[TL_{d-1}^{L,U}]\right)}$$

This equation defines the CDO fair value, but can also be used to calibrate the implied correlation parameter from the market.
This can be obtained by plugging into the equation the market premium value and solve for the correlation parameter $\rho$.

As usual let's implement a $\tt{python}$ class to represent CDOs.

In [53]:
from finmarkets import DiscountCurve, CreditCurve, generate_swap_dates
from scipy.integrate import quad
from scipy.stats import norm, binom
import numpy as np
from numpy import exp, sqrt
from datetime import date
from dateutil.relativedelta import relativedelta

class CollDebtObligation:
    def __init__(self, notional, names, tranches, rho, cc,
                 start_date, spreads,
                 maturity, tenor=3, recovery=0.4):
        self.notional = notional
        self.names = names
        self.tranches = tranches
        self.payment_dates = generate_swap_dates(start_date, maturity * 12, tenor)
        self.spreads = spreads
        self.rho = rho
        self.recovery = recovery
        self.cc = cc

    def one_factor_model(self, M, Q, l, L, U):
        P = norm.cdf((norm.ppf(Q) - sqrt(self.rho) * M) / (sqrt(1 - self.rho)))
        b = binom(self.names, P)
        return b.pmf(l) * norm.pdf(M) * max(min(l/self.names * 
                                                self.notional * 
                                                (1 - self.recovery), U) - L, 0)
        
    def expected_tranche_loss(self, d, L, U):
        Q = 1 - self.cc.ndp(d)
        v = 0
        for l in range(self.names+1):
            i = quad(self.one_factor_model, -np.inf, np.inf, 
                     args=(Q, l, L, U))[0]
            v += i
        return v

    def npv_premium(self, tranche, dc):
        L = self.tranches[tranche][0] * self.notional
        U = self.tranches[tranche][1] * self.notional
        v = 0
        for i in range(1, len(self.payment_dates)):
            ds = self.payment_dates[i - 1]
            de = self.payment_dates[i]
            D = dc.df(de)
            ETL = self.expected_tranche_loss(ds, L, U)
            v += D * (de - ds).days / 360 * max((U - L) - ETL, 0)
        return v * self.spreads[tranche]

    def npv_default(self, tranche, dc):
        U = self.tranches[tranche][1] * self.notional
        L = self.tranches[tranche][0] * self.notional
        v = 0
        for i in range(1, len(self.payment_dates)):
            ds = self.payment_dates[i - 1]
            de = self.payment_dates[i]
            ETL1 = self.expected_tranche_loss(ds, L, U)
            ETL2 = self.expected_tranche_loss(de, L, U)
            v += dc.df(de) * (ETL2 - ETL1)
        return v

    def npv(self, tranche, dc):
        return self.npv_default(tranche, dc) - self.npv_premium(tranche, dc)

    def fair_value(self, tranche, dc):
        num = self.npv_default(tranche, dc)
        den = self.npv_premium(tranche, dc) / self.spreads[tranche]
        return num / den

In [54]:
pillar_dates = []
df = []
observation_date = date.today()

for i in range(2):
    pillar_dates.append(observation_date + relativedelta(years=i))
    df.append(1 / (1 + 0.05) ** i)
dc = DiscountCurve(observation_date, pillar_dates, df)

cc = CreditCurve([observation_date + relativedelta(years=i) for i in range(5)],
                 [1, 0.99, 0.97, 0.95, 0.93])

tranches = [[0.0, 0.03], [0.03, 0.06], [0.06, 0.09], [0.09, 1.0]]
spreads = [0.15, 0.07, 0.03, 0.01]

cdo = CollDebtObligation(100e6, 125, tranches, 0.3, cc,
                         observation_date, spreads, 1, 12)
for i in range(len(tranches)):
    print ("Tranche {} ({}): {:.5f}".format(i, tranches[i], cdo.fair_value(i, dc)))

Tranche 0 ([0.0, 0.03]): 0.15942
Tranche 1 ([0.03, 0.06]): 0.02505
Tranche 2 ([0.06, 0.09]): 0.00773
Tranche 3 ([0.09, 1.0]): 0.00017


In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta
from finmarkets import DiscountCurve, CreditCurve, generate_swap_dates

T = 3 
C = 0.06
r = 0.03
R = 0.4
N = 100

obs_date = date.today()
pillars = [obs_date+relativedelta(years=i) for i in range(T+1)]
dfs = [1/(1+r)**i for i in range(T+1)]
dc = DiscountCurve(obs_date, pillars, dfs)
S = [1, 0.9, 0.8, 0.7]
cc = CreditCurve(pillars, S)

cva = 0
d = obs_date
while d <= pillars[-1]:
    cva += dc.df(d)*(cc.ndp(d) - cc.ndp(d+relativedelta(days=1)))
    d += relativedelta(days=1)

cva = cva * (1-R) * N
print ("CVA: {:.2f}".format(cva))

In [ ]:
default_free_price = 0
for i in range(1, len(pillars)):
    default_free_price += (N * ((pillars[i] - pillars[i-1]).days / 360) * 
          dc.df(pillars[i]) * C )
                    
default_free_price += N * dc.df(pillars[-1])
             
print ("Bond Price: {:.2f}".format(default_free_price - cva))